In [3]:
import torch
bart = torch.hub.load('pytorch/fairseq', 'bart.large')
bart.eval()
bart.cuda()

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /root/.cache/torch/hub/master.zip


running build_ext
cythoning fairseq/data/data_utils_fast.pyx to fairseq/data/data_utils_fast.cpp


/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:369: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))


cythoning fairseq/data/token_block_utils_fast.pyx to fairseq/data/token_block_utils_fast.cpp
building 'fairseq.libbleu' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/fairseq
creating build/temp.linux-x86_64-3.7/fairseq/clib
creating build/temp.linux-x86_64-3.7/fairseq/clib/libbleu
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-OGiuun/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-OGiuun/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c fairseq/clib/libbleu/libbleu.cpp -o build/temp.linux-x86_64-3.7/fairseq/clib/libbleu/libbleu.o -std=c++11 -O3 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE="_gcc" -DPYBIND11_STDLIB="_libstdcpp" -DPYBIND11_BUILD_ABI="_cxxabi1011" -DTORCH_EXTENSION_

100%|██████████| 3699866548/3699866548 [01:36<00:00, 38152789.96B/s]
1042301B [00:00, 5543557.75B/s]
456318B [00:00, 4162504.92B/s]


BARTHubInterface(
  (models): ModuleList(
    (0): BARTModel(
      (encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDropo

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
df = pd.read_csv('/content/df_aae_full_mask.csv')

In [5]:
df.sample(10)

,Unnamed: 0,index,local_sufficency,TEXT
463,463,essay182_1,1,"First, <mask>. It would divert public funds from other needs such as law enforcement, infrastructure and so forth. There are many other needs which should come first, such as providing health care to all American, or improving public schools. In addition, it would cause to raise our taxes again and again to fund college education. Consequently, it would affect all taxpayers whether or not they benefit from the use of the funds."
12,12,essay006_2,1,"One other important factor is <mask>. For example, they will get exposed to a different educational system. They will meet new professors and new classmates which makes the academic experience different from that in their home country."
761,761,essay290_2,1,"Second, I think that <mask>. The fact that a company is a large one, this means that it's having a considerable success. In other words, this means that the company can give more salaries to its employees. For instance, common sense would tell us that a famous and international known company won't give the same salary as a small local company."
995,995,essay389_1,1,"In a developing country, not many can afford to send their children to school. There are government schools where tuition fees are practically free but they are not aplenty and mostly are situated in the cities. They are not reachable by children who live in the mountains or islands. These children whose parents are either farmers or fishermen, can hardly eat three meals a day, let alone afford to pay for transportation to get to these schools everyday. As a result, <mask>."
629,629,essay243_1,0,"Everyone agrees that <mask>. You can talk to others from thousands mile. For instance, if you are in Vietnam and another person is in the US - half of the Earth, no matter how far it is, you can keep contact. We will never have to worry if our mails will be sent to the right address. If the address is wrong, then you will know right away. Moreover, e-mail doesn't take too much time: just a click."
745,745,essay284_3,1,"In fact, those good endings sometimes are helpful. Some people may be encouraged to do good things. But like I said, this kind of behavior won’t last long, because someday they will realize the truth. So I suggest <mask>. And if they can, they will be good people no matter how the story ends."
884,884,essay338_1,1,"Mentioned to societies life style, they constantly use these kinds of electronic letter for making fellowship or just continuing theirs what they already had. Obviousness, <mask>."
757,757,essay289_1,1,"To begin with, Evaluating teachers made by students could improve teachers' performance. For example, evaluation made by students is similar to feedback from customer that business owners need to know in order to improve their product. Students might complain about their teachers spoke too fast that students could not catch up on information. Consequently, teachers could improve themselves by speaking slower to make sure that students could get all information that they want to give. Therefore, <mask>."
562,562,essay217_3,0,"According to myself, both ways are good, but I prefer the latter. It is because growth is a long process that experience is indispensable. I believe that <mask>. Parents and other predecessors used to experience their own life by themselves, why don't the younger try to do the same?"
182,182,essay073_1,0,"Firstly, <mask>. For example, nowadays horrendous images are displayed on the cigarette boxes to illustrate the consequences of smoking. As a result, statistics show a slight reduction in the number of smokers, indicating that they realize the effects of the negative habit. Not only that, the magnificent photograph captured by Kevin Carter, which portrayed a starving Sudanese child struck by extreme poverty has successfully highlighted the plight faced by the citizens in Sudan. Apart from that, images are also widely used in newspapers, magazines and advertisements. T

In [75]:
txt = """Tax revenue is collected from those who works. Therefore <mask> I think. Spending them on helping those not make contribution to society would not sound fair to tax payers. Besides, there are other pressing demands on the government expenditure, such as health care service and education, so addressing unemployment should have low priority."""

In [76]:
filled_para_text = bart.fill_mask([txt], topk=1, beam=5, match_source_len=False)[0][0][0]
print(filled_para_text)

I don’t think so. Tax revenue is collected from those who works. Spending them on helping those not make contribution to society would not sound fair to tax payers. Besides, there are other pressing demands on the government expenditure, such as health care service and education, so addressing unemployment should have low priority. Therefore I don’t think it’s a good idea, I think.


In [77]:
from tqdm import tqdm
pred_conclusions = []
filled_para_texts = []
for i in range(len(df)):
    print('SOURCE:')
    #print(df['masked_para_text'][i])
    #print('-'*50)
    #print('TARGET FILL:')
    #print(df['conclusion'][i])
    #print('SUFFICIENT:')
    if df['TEXT'][i][0] == '<':
    txt = 'I think that ' + df['TEXT'][i]
    elif '. <mask>.' in df['TEXT'][i][0]:
    txt = df['TEXT'][i].replace('. <mask>.', '. I think that <mask>.')
    else:
    txt = df['TEXT'][i]
    print(df['TEXT'][i])
    #print('-'*50)
    print('PRED FILL:')
    filled_para_text = bart.fill_mask([txt.replace('<mask>', '<mask> ')], topk=1, beam=5, match_source_len=False)[0][0][0]
    print(filled_para_text)
    #split_para_text = df['full_para_text'][i].split(df['conclusion'][i])
    #pred_conclusion = filled_para_text.replace(split_para_text[0], '')
    #if len(split_para_text[1])>5:
    #    pred_conclusion = re.sub('('+split_para_text[1][0:10]+').*', '', pred_conclusion)
    #print(pred_conclusion)
    #pred_conclusions.append(pred_conclusion)
    filled_para_texts.append(filled_para_text)
    #print('-'*50)
    print('\n')

Streaming output truncated to the last 5000 lines.


SOURCE:
Admittedly, the government should give priority to public services and facilities. This is because the government plays the role of a leader among citizens, which should discharge the responsibility of making optimal use of the tax money to keep society safe and stable. Simultaneously, public services such as education system or medical care can meet people's basic demand of survival, security and knowledge, thus making them feel comfortable, which can lower the crime rate. Therefore, <mask>.
PRED FILL:
Admittedly, the government should give priority to public services and facilities. This is because the government plays the role of a leader among citizens, which should discharge the responsibility of making optimal use of the tax money to keep society safe and stable. Simultaneously, public services such as education system or medical care can meet people's basic demand of survival, security and knowledge, thus making them f

In [84]:
import re

In [80]:
df['TEXT_filled'] = filled_para_texts

In [81]:
df.to_csv('/content/df_aae_full_mask_filled.csv')

In [97]:
re.search('(<mask>).*\.', df['TEXT'][1019])[0][5:].split('.')[0]

'>, and this could avoid imbalance of gender in some subjects'

In [99]:
df['TEXT_filled'][1019][df['TEXT'][1019].find('<mask>'):].split('.')[0].replace(re.search('(<mask>).*\.', df['TEXT'][1019])[0][6:].split('.')[0], '')

'it is possible to have a single gender teacher '

In [110]:
replacements = []
for i in range(len(df)):
    if '. <mask>.' in df['TEXT'][i]:
        replacement = df['TEXT_filled'][i].split('.')[-2].strip()
    elif df['TEXT'][i] == '<mask>':
        replacement = '<mask>'
    else:
        replacement = df['TEXT_filled'][i][df['TEXT'][i].find('<mask>'):].split('.')[0].replace(re.search('(<mask>).*\.', df['TEXT'][i])[0][6:].split('.')[0], '')
    if replacement != '<mask>' and replacement in df['TEXT'][i]:
        replacement = df['TEXT_filled'][i].split('.')[-2].strip()
    if replacement != '<mask>' and replacement in df['TEXT'][i]:
        for sentence in df['TEXT_filled'][i].split('.'):
            if sentence not in df['TEXT'][i].split('.'):
                replacement=sentence
    if replacement != '<mask>' and replacement in df['TEXT'][i] and '. .' not in df['TEXT_filled'][i]:
    print(df['TEXT'][i])
    print(df['TEXT_filled'][i])
    print(replacement)
    print(i)
    print('-'*50)
    replacements.append(replacement)

Admittedly, on the other hand, there might be some people proclaiming that <mask> because it is not necessary for them to notice the time control or acquire the fame of succeed. However, from my limited understanding, I just doubt the credibility of that saying. I do not think that those people with slower pace losing the piano competition can be really happy. In contrast, it is really joyful for me that I can immerse in the joy of the victory and accomplishment. 
Admittedly, on the other hand, there might be some people proclaiming that those people with slower pace losing the piano competition can be really happy because it is not necessary for them to notice the time control or acquire the fame of succeed. However, from my limited understanding, I just doubt the credibility of that saying. I do not think that those people with slower pace losing the piano competition can be really happy. In contrast, it is really joyful for me that I can immerse in the joy of the victory and accompl

In [111]:
df['replacement'] = replacements

In [112]:
df.to_csv('/content/df_aae_full_mask_filled.csv')